In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import qmc
from scipy.spatial.distance import cdist 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.neighbors import NearestNeighbors
import warnings
import random

# --- Configuration ---
# Suppress heavy TF logs and warnings for cleaner output
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

print("Libraries loaded successfully.")

In [ ]:
# Load your specific function data here.
# Ensure these files exist in the directory you are running the notebook from.

week = 'week13' 
# Adjust this path if your folder structure is different in the repo
base_path = './Capstone_Data/' + week + '_data/function_8/'

try:
    data_x = np.load(base_path + 'initial_inputs.npy')
    data_y = np.load(base_path + 'initial_outputs.npy')
    print(f"Data Loaded Successfully: X shape {data_x.shape}, Y shape {data_y.shape}")
except FileNotFoundError:
    print(f"Error: Could not find .npy files at {base_path}")
    print("Please check your directory structure.")

In [ ]:
--- Helper Function: Latin Hypercube Sampling ---
def get_latin_hypercube_samples(bounds, n_samples, seed):
    rng = np.random.RandomState(seed)
    seed_val = rng.randint(0, 10000)
    sampler = qmc.LatinHypercube(d=len(bounds), optimization="random-cd", seed=seed_val)
    samples = sampler.random(n_samples)
    l_bounds = bounds[:, 0]
    u_bounds = bounds[:, 1]
    samples_scaled = qmc.scale(samples, l_bounds, u_bounds)
    return samples_scaled

# --- 1. Define Keras Model ---
def create_model(n_dim, learning_rate=1e-3):
    model = Sequential([
        Input(shape=(n_dim,)),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1)
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# --- 2. Deep Ensemble Class ---
class DeepEnsemble:
    def __init__(self, n_models, n_dim):
        self.n_models = n_models
        self.n_dim = n_dim
        self.models = [create_model(n_dim) for _ in range(n_models)]

    def fit(self, X, y, epochs, batch_size, validation_split=0.2, verbose=0):
        # Callbacks for robust training
        early_stopper = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
        callbacks = [early_stopper, lr_scheduler]

        for i, model in enumerate(self.models):
            # Bagging: Sample with replacement to induce diversity
            indices = np.random.choice(len(X), size=len(X), replace=True)
            X_sample = X[indices]
            y_sample = y[indices]

            model.fit(X_sample, y_sample, epochs=epochs, batch_size=batch_size,
                      shuffle=True, verbose=verbose, validation_split=validation_split,
                      callbacks=callbacks)

    def predict(self, X_test):
        all_preds = [model.predict(X_test, verbose=0) for model in self.models]
        stacked_preds = np.stack(all_preds, axis=0)
        mu = np.mean(stacked_preds, axis=0)
        std = np.std(stacked_preds, axis=0)
        return mu, std

In [ ]:
# --- 3. BBO Function with PCA and Spatially Aware Filtering ---
def BBO_Ensemble_PCA(X, Y, beta, n_candidates, seed, n_models, epochs, batch_size):
    """
    Propose next point using Deep Ensemble with Optional PCA and 
    spatially-aware acquisition penalties.
    """
    X = np.asarray(X, dtype=float)
    Y = np.asarray(Y, dtype=float).reshape(-1, 1)
    n_dim = X.shape[1]

    # --- PCA LOGIC ---
    # Apply PCA only if dimensions are high (e.g., >= 5) and sufficient data exists
    use_pca = (n_dim >= 5) and (X.shape[0] > n_dim)

    if use_pca:
        n_components = 3
        print(f"Applying PCA: Reducing {n_dim}D -> {n_components}D")
        pca = PCA(n_components=n_components)
        X_transformed = pca.fit_transform(X)
        opt_dim = n_components
    else:
        print(f"Skipping PCA: optimizing in original {n_dim}D space")
        X_transformed = X
        opt_dim = n_dim

    # 1. Scale Data
    x_scaler = StandardScaler()
    X_scaled = x_scaler.fit_transform(X_transformed)
    y_scaler = StandardScaler()
    Y_scaled = y_scaler.fit_transform(Y)

    # 2. Train Ensemble
    print(f"Training Deep Ensemble ({n_models} models)...")
    ensemble = DeepEnsemble(n_models=n_models, n_dim=opt_dim)
    ensemble.fit(X_scaled, Y_scaled, epochs=epochs, batch_size=batch_size)

    # 3. Generate Candidate Grid
    if use_pca:
        # Sample in latent space (Gaussian)
        rng = np.random.default_rng(seed)
        grid_latent = rng.standard_normal(size=(n_candidates, opt_dim))
        grid_scaled = grid_latent 
    else:
        # Sample in original space (Latin Hypercube)
        bounds = np.array([[0.0, 1.0]] * n_dim)
        grid = get_latin_hypercube_samples(bounds, n_candidates, seed)
        grid_scaled = x_scaler.transform(grid)

    # 4. Predict
    mu_scaled, std_scaled = ensemble.predict(grid_scaled)

    # 5. Inverse Scale Predictions
    mu = y_scaler.inverse_transform(mu_scaled)
    std = std_scaled * y_scaler.scale_

    # 6. Calculate UCB Acquisition
    ucb = mu.ravel() + beta * std.ravel()

    # --- Spatially Aware Filtering ---
    # A. Domain Constraints
    if use_pca:
        grid_original_space = pca.inverse_transform(x_scaler.inverse_transform(grid_scaled))
        # Discard points outside valid bounds [0,1]
        mask = np.all((grid_original_space >= 0.0) & (grid_original_space <= 1.0), axis=1)
        
        if np.sum(mask) > 100:
            candidates_X = grid_original_space[mask]
            ucb = ucb[mask]
        else:
            candidates_X = np.clip(grid_original_space, 0.0, 1.0)
    else:
        candidates_X = grid

    # B. Local Penalties (Penalize regions known to be poor)
    nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(X)
    distances, indices = nbrs.kneighbors(candidates_X)
    
    neighbor_Y = Y[indices].squeeze() 
    local_mean_Y = np.mean(neighbor_Y, axis=1)
    global_mean_Y = np.mean(Y)
    
    # Punishment factor for bad neighborhoods
    scaling_factor = 2.0
    penalty = np.maximum(0, global_mean_Y - local_mean_Y) * scaling_factor
    ucb_adjusted = ucb - penalty

    # C. "Too Close" Filter (Prevent resampling)
    min_dist_to_known = np.min(distances, axis=1)
    ucb_adjusted[min_dist_to_known < 1e-4] = -np.inf

    # 7. Select Best
    best_idx = int(np.argmax(ucb_adjusted))
    x_next = candidates_X[best_idx]

    return x_next

In [ ]:
# --- Execution ---

# Parameters
BETA = 0.05        # Low exploration weight for exploitation
SEARCH_SIZE = 10000
N_MODELS = 20      # Number of models in ensemble
EPOCHS = 250       
BATCH_SIZE = 32

# Note: Ensure data_x and data_y are loaded in Cell 2 before running this!
if 'data_x' in locals() and 'data_y' in locals():
    print(f"--- Running Optimization on Function ---")
    next_point = BBO_Ensemble_PCA(data_x, data_y, BETA, SEARCH_SIZE, SEED,
                                   n_models=N_MODELS,
                                   epochs=EPOCHS,
                                   batch_size=BATCH_SIZE)

    # Formatting Output
    next_point_rounded = np.round(next_point, 6)
    print("-" * 50)
    print(f"Next Query Point Suggested: \n{next_point_rounded}")
    print("-" * 50)
else:
    print("Error: data_x and data_y not defined. Please check Cell 2.")